## Groceries Recommendation System


#### Load data 

In [1]:
import pandas as pd 
import timeit

df_orders = pd.read_csv('./data/orders.csv')
df_aisles = pd.read_csv('./data/aisles.csv')
df_departments = pd.read_csv('./data/departments.csv')
df_order_products = pd.read_csv('./data/order_products__prior_10000000.csv')
df_products = pd.read_csv('./data/products.csv')


In [2]:
df_orders.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


#### Print order sample

In [3]:
df_order_products.head(5)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


#### Print products sample

In [4]:
df_products.head(5)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [5]:
# get all unique products 
unique_products_df = df_order_products[['product_id']].groupby('product_id')

unique_products_with_counts_df = pd.DataFrame(unique_products_df.size().reset_index(name = "group_count"))
# filter out the the product if it does not appear in more than 500 orders out of the 3 million

unique_products_filtered = unique_products_with_counts_df[unique_products_with_counts_df.group_count > 500]

unique_products_filtered = unique_products_filtered.set_index('product_id')
print(unique_products_filtered.size, "Products")
unique_products_filtered.head(5)

3377 Products


,group_count
product_id,
1,567
10,794
25,639
34,1987
45,5711


In [6]:
print(df_order_products.size)
df_order_products.head(5)

39999996


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [7]:
df_order_and_products = df_order_products.join(unique_products_filtered, 
                                               on='product_id', 
                                               how='inner')
df_order_and_products.head(5)

,order_id,product_id,add_to_cart_order,reordered,group_count
0,2,33120,1,1,6069
237,26,33120,5,0,6069
1084,120,33120,13,0,6069
3039,327,33120,5,1,6069
3709,390,33120,28,1,6069


In [8]:
df_order_and_products_tuple = df_order_and_products[['order_id', 'product_id']]

# quick filter for testing
#df_order_and_products_tuple = df_order_and_products_tuple[(df_order_and_products_tuple.order_id < 500)]
df_order_and_products_tuple.head(5)

,order_id,product_id
0,2,33120
237,26,33120
1084,120,33120
3039,327,33120
3709,390,33120


In [9]:
print(df_order_and_products_tuple.size)
df_order_and_products_tuple.head(5)

15037848


,order_id,product_id
0,2,33120
237,26,33120
1084,120,33120
3039,327,33120
3709,390,33120


In [10]:
# get product count for each order
productCounts = df_order_and_products_tuple[['order_id']].groupby('order_id')
productCounts = productCounts.size().reset_index(name = "group_count")

print(df_order_and_products_tuple.head(10))

# only get orders that have move thant 3 orders
productCounts = productCounts[(productCounts[['group_count']].group_count > 3)].drop(columns=['group_count'])
print(df_order_and_products_tuple.size)

df_order_and_products_tuple = pd.merge(df_order_and_products_tuple, productCounts, on=['order_id', 'order_id'], how='inner')


      order_id  product_id
0            2       33120
237         26       33120
1084       120       33120
3039       327       33120
3709       390       33120
5055       537       33120
5544       582       33120
5838       608       33120
5986       623       33120
6635       689       33120
15037848


In [11]:
df_order_and_products_with_names = df_order_and_products_tuple.merge(df_products, on='product_id', how='inner')
# df_order_and_products_with_names
df_order_and_products_names = df_order_and_products_with_names[['order_id','product_name']]
df_order_and_products_names.head(5)

#oneHotEncMatrix = pd.get_dummies(df_order_and_products_names, columns=['product_name'], sparse=True)
#print(oneHotEncMatrix)
#df = df_order_products3.reset_index().pivot(index='index', columns='order_id', values='product_id')


,order_id,product_name
0,2,Organic Egg Whites
1,26,Organic Egg Whites
2,120,Organic Egg Whites
3,327,Organic Egg Whites
4,390,Organic Egg Whites


In [12]:
# dfanew = sparse_ohe(df_order_and_products_names, 'product_name', vals)
df_order_and_products_names.head(5)

,order_id,product_name
0,2,Organic Egg Whites
1,26,Organic Egg Whites
2,120,Organic Egg Whites
3,327,Organic Egg Whites
4,390,Organic Egg Whites


### Group the orders to products as a list

In [13]:
grouped_orders_to_product_list = df_order_and_products_names.groupby('order_id')['product_name'].apply(list)
grouped_orders_to_product_list = grouped_orders_to_product_list.to_frame()
grouped_orders_to_product_list.to_csv("./output/grouped_orders_to_product_list.csv", index=False)
grouped_orders_to_product_list.head(10)

,product_name
order_id,
2,"[Organic Egg Whites, Michigan Organic Kale, Ga..."
3,"[Organic Baby Spinach, Total 2% with Strawberr..."
4,"[Original Orange Juice, Energy Drink, Nutri-Gr..."
5,"[Organic Hass Avocado, Bag of Organic Bananas,..."
9,"[Distilled Water, Organic Red Radish, Bunch, E..."
10,"[Banana, Organic Avocado, Organic Cilantro, Or..."
11,"[Extra Virgin Olive Oil, Traditional Refried B..."
12,"[Cherubs Heavenly Salad Tomatoes, Ruby Red Gra..."
13,"[Sparkling Natural Mineral Water, Lemonade, Ph..."


In [14]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
oneHotEncMatrix = grouped_orders_to_product_list.join(pd.DataFrame(mlb.fit_transform(grouped_orders_to_product_list.pop('product_name')),
                          columns=mlb.classes_,
                          index=grouped_orders_to_product_list.index))


In [15]:
# oneHotEncMatrix.to_csv("oneHotEncMatrix.csv", index=False)
oneHotEncMatrix.head(10)

,0% Fat Free Organic Milk,0% Fat Organic Greek Vanilla Yogurt,0% Greek Strained Yogurt,1 Liter,1 Ply Paper Towels,1% Low Fat Milk,1% Lowfat Milk,1% Milkfat Low Fat Buttermilk,100 Calorie Per Bag Popcorn,100% Apple Juice,...,Zero Calorie Sweetener,Zero Go-Go Mixed Berry Vitamin Water,Zero Soda,Zero Vitamin Water,Zucchini Noodles,Zucchini Squash,of Hanover 100 Calorie Pretzels Mini,smartwater® Electrolyte Enhanced Water,vitaminwater® XXX Acai Blueberry Pomegranate,with Crispy Almonds Cereal
order_id,,,,,,,,,,,,,,,,,,,,,
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calulate One-Hot-Encoding (VERY Expensive operation don't use) Move efficient way above 

In [16]:
# oneHotEncMatrix =  pd.get_dummies(df_order_and_products_names, columns=['product_name'], sparse =False)
# oneHotEncMatrix.head(10)
# oneHotEncMatrix.to_csv("oneHotEncMatrix_not_grouped.csv", index=False)

In [17]:
# print(type(oneHotEncMatrix))
# oneHotEncMatrix = oneHotEncMatrix.set_index('order_id', inplace=True)
# print(type(oneHotEncMatrix))

In [18]:
# oneHotEncMatrix = oneHotEncMatrix.groupby(['order_id'], as_index=False).max()
# print(oneHotEncMatrix.size)
# oneHotEncMatrix.head(15)
# oneHotEncMatrix.to_csv("oneHotEncMatrix_grouped.csv", index=False)

In [19]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(oneHotEncMatrix, min_support=0.01, use_colnames=True)
frequent_itemsets.to_csv("./output/frequent_itemsets.csv", index=False)

#### Frequent Itemsets

In [20]:
frequent_itemsets.sort_values(by=['support'], ascending=False).head(5)

,support,itemsets
7,0.191068,(Banana)
6,0.150365,(Bag of Organic Bananas)
102,0.109141,(Organic Strawberries)
48,0.099042,(Organic Baby Spinach)
74,0.088798,(Organic Hass Avocado)


In [21]:
from mlxtend.frequent_patterns import association_rules

association_rules_df = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
association_rules_df.to_csv("./output/association_rules_df.csv", index=False)

#### Association Rules

In [22]:
association_rules_df.sort_values(by=['support'], ascending=False).head(8)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(Organic Hass Avocado),(Bag of Organic Bananas),0.088798,0.150365,0.026621,0.299790,1.993752,0.013269,1.213401
5,(Bag of Organic Bananas),(Organic Hass Avocado),0.150365,0.088798,0.026621,0.177041,1.993752,0.013269,1.107226
10,(Organic Strawberries),(Bag of Organic Bananas),0.109141,0.150365,0.026482,0.242644,1.613700,0.010071,1.121843
11,(Bag of Organic Bananas),(Organic Strawberries),0.150365,0.109141,0.026482,0.176120,1.613700,0.010071,1.081298
30,(Organic Strawberries),(Banana),0.109141,0.191068,0.024239,0.222091,1.162369,0.003386,1.039881
31,(Banana),(Organic Strawberries),0.191068,0.109141,0.024239,0.126862,1.162369,0.003386,1.020296
24,(Organic Avocado),(Banana),0.073232,0.191068,0.022734,0.310446,1.624793,0.008742,1.173123
25,(Banana),(Organic Avocado),0.191068,0.073232,0.022734,0.118986,1.624793,0.008742,1.051934


In [23]:
association_rules_df[association_rules_df['antecedents'] == {'Organic Whole Milk'}]


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,(Organic Whole Milk),(Bag of Organic Bananas),0.055789,0.150365,0.011263,0.201893,1.342685,0.002875,1.064563
32,(Organic Whole Milk),(Banana),0.055789,0.191068,0.013587,0.243541,1.274632,0.002927,1.069367
59,(Organic Whole Milk),(Organic Strawberries),0.055789,0.109141,0.010360,0.185699,1.701461,0.004271,1.094017


### Recommendation function

In [24]:
def recommend_product(product_name, association_rules_df, topN):
    print("Searching recommnedation for: ", set(product_name))
    rec = association_rules_df[association_rules_df['antecedents'] == product_name]
    return rec.head(topN)


In [25]:
recommend_product({"Organic Whole Milk"}, association_rules_df, 5)

Searching recommnedation for:  {'Organic Whole Milk'}


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
12,(Organic Whole Milk),(Bag of Organic Bananas),0.055789,0.150365,0.011263,0.201893,1.342685,0.002875,1.064563
32,(Organic Whole Milk),(Banana),0.055789,0.191068,0.013587,0.243541,1.274632,0.002927,1.069367
59,(Organic Whole Milk),(Organic Strawberries),0.055789,0.109141,0.010360,0.185699,1.701461,0.004271,1.094017


In [26]:
recommend_product({'Organic Zucchini'}, association_rules_df, 5)

Searching recommnedation for:  {'Organic Zucchini'}


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


In [27]:
recommend_product({'Organic Avocado'}, association_rules_df, 5)

Searching recommnedation for:  {'Organic Avocado'}


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
24,(Organic Avocado),(Banana),0.073232,0.191068,0.022734,0.310446,1.624793,0.008742,1.173123
40,(Organic Avocado),(Large Lemon),0.073232,0.063481,0.010596,0.144688,2.279227,0.005947,1.094944
42,(Organic Avocado),(Organic Baby Spinach),0.073232,0.099042,0.013274,0.181256,1.830090,0.006021,1.100414
44,(Organic Avocado),(Organic Strawberries),0.073232,0.109141,0.010253,0.140004,1.282786,0.002260,1.035888


In [28]:
recommend_product({'Organic Hass Avocado'}, association_rules_df, 5)

Searching recommnedation for:  {'Organic Hass Avocado'}


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
4,(Organic Hass Avocado),(Bag of Organic Bananas),0.088798,0.150365,0.026621,0.299790,1.993752,0.013269,1.213401
47,(Organic Hass Avocado),(Organic Baby Spinach),0.088798,0.099042,0.014939,0.168240,1.698680,0.006145,1.083195
53,(Organic Hass Avocado),(Organic Raspberries),0.088798,0.056490,0.010942,0.123220,2.181281,0.005925,1.076108
55,(Organic Hass Avocado),(Organic Strawberries),0.088798,0.109141,0.017657,0.198843,1.821892,0.007965,1.111965


In [29]:
recommend_product({'Organic Raspberries'}, association_rules_df, 5)


Searching recommnedation for:  {'Organic Raspberries'}


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
8,(Organic Raspberries),(Bag of Organic Bananas),0.05649,0.150365,0.017318,0.306570,2.038839,0.008824,1.225264
52,(Organic Raspberries),(Organic Hass Avocado),0.05649,0.088798,0.010942,0.193693,2.181281,0.005925,1.130093
57,(Organic Raspberries),(Organic Strawberries),0.05649,0.109141,0.014520,0.257041,2.355130,0.008355,1.199069


## Final code stops here, the rest of the code below is testing an references

In [ ]:
#oneHotEncMatrix = pd.get_dummies(df_order_products3, columns=['product_name'], sparse=True)
#print(oneHotEncMatrix)
#df = df_order_products3.reset_index().pivot(index='index', columns='order_id', values='product_id')

In [ ]:
# import numpy as np
# from sklearn.preprocessing import MultiLabelBinarizer


# mlb = MultiLabelBinarizer()
# df1 = pd.DataFrame(mlb.fit_transform(df_order_products3['product_name']),
#                    columns=[mlb.classes_], 
#                    index=df_order_products3.order_id)


In [ ]:
oneHotEncMatrix = pd.get_dummies(df_order_products3, 
                                 columns=['product_name']).groupby(['order_id'], 
                                                                   as_index=False).max()
oneHotEncMatrix.head(5)

In [ ]:
#oneHotEncMatrix = pd.get_dummies(df_order_products3, columns=['product_id'], sparse=True)
#oneHotEncMatrix = oneHotEncMatrix.groupby('order_id').agg('sum')

In [ ]:
oneHotEncMatrix.head(10)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
#df2 = cosine_similarity(oneHotEncMatrix.head(1000))


In [ ]:
oneHotEncMatrix.head(10)

basket_sets = oneHotEncMatrix
basket_sets = basket_sets.drop(columns=['order_id'])


In [ ]:
print(basket_sets)

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)


In [ ]:
basket_sets.head(10)

In [ ]:
frequent_itemsets

In [ ]:
frequent_itemsets.head(10)

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=.01)
rules

In [ ]:
rules.to_csv("./output/rules.csv", index=False)

In [ ]:
print(df_products.loc[4604])
print()
print(df_products.loc[21902])

In [ ]:
print(df_products.loc[4604])
print()
print(df_products.loc[24851])

In [ ]:
df_orders = pd.read_csv('./data/orders.csv')
df_aisles = pd.read_csv('./data/aisles.csv')
df_departments = pd.read_csv('./data/departments.csv')
df_order_products = pd.read_csv('./data/order_products__prior.csv')
df_products = pd.read_csv('./data/products.csv')

# get all unique products 
unique_products_df = df_order_products[['product_id']].groupby('product_id')

# group all products to get the counts
unique_products_with_counts_df = pd.DataFrame(unique_products_df.size().reset_index(name = "group_count"))

# filter out the the product if it does not appear in more than 1,000 orders out of the 3 million
unique_products_filtered = unique_products_with_counts_df[unique_products_with_counts_df.group_count > 1000]

unique_products_filtered = unique_products_filtered.set_index('product_id')



In [ ]:
# import numpy as np
# import pandas as pd
# import scipy.sparse as ssp

# from dummyPy import OneHotEncoder

# vals =  np.array(list(set(df_order_and_products_names.product_name.unique())))
# #print(vals)
# # data = pd.read_csv("titanic.csv",
# #                    usecols=["Pclass", "Sex", "Age", "Fare", "Embarked"])

# # train_data = data[data["Embarked"] != "S"]
# # test_data = data[data["Embarked"] == "S"]

# encoder = OneHotEncoder(['order_id'])

# encoder.fit_transform(df_order_and_products_names)

# print(encoder.unique_vals)